# MarketPlace data sink app and reaxpro platform interfacing



*If you operating on **snellius**, follow [this documentation](https://servicedesk.surf.nl/wiki/pages/viewpage.action?pageId=30660252#JupyterNotebooksonSnellius-UsingPythonvirtualenvironmentswithSnellius'JupyterHub) for setting up you virtual environment with the needed modules and packges.*

Run the following `bash`-commands in order to setup the jupyter environment:

```
# Load Python module used by the https://jupyter.snellius.surf.nl/2021 JupyterHub
module load 2022
module load Python/3.10.4-GCCcore-11.3.0
 
# Create and activate virtual environment
virtualenv marketplace_env
 
# Purge modules so that any subsequent pip-installs don't pick up on python packages from the module environment
module purge
 
# Activate virtual environment
source marketplace_env/bin/activate

# make sure that you are using the pip module from the correct python kernel
which pip
  
# Install ipykernel in the virtual environment
pip install ipykernel reaxpro-workflow-service git+https://github.com/materials-marketplace/python-sdk.git@fix/import-error

# make sure that you are using the correct python kernel
which python3

# Install the virtual environment as custom kernel. It will show up in the Jupyter Notebook Server with the name passed to the '--name' argument.
python3 -m ipykernel install --user --name=marketplace_env
 
# Makes sure the kernel only uses Python packages from the conda environment, not from the module environment
sed -i '/"-m",/i \ \ "-E",' ~/.local/share/jupyter/kernels/marketplace_env/kernel.json
```

Now, navigate to `https://jupyter.snellius.surf.nl/2022/` and log in with your credentials.

First you follow the previous steps, you are able to operate with this noteboke. 

Make sure that the needed packages are installed, if you did not already in the previous steps.

In [15]:
!pip install reaxpro-workflow-service git+https://github.com/materials-marketplace/python-sdk.git@fix/import-error

Defaulting to user installation because normal site-packages is not writeable
  Using cached marketplace_sdk-0.5.0-py3-none-any.whl (20 kB)
  Using cached rdflib_jsonld-0.6.2-py2.py3-none-any.whl (4.0 kB)
  Using cached rdflib-6.2.0-py3-none-any.whl (500 kB)
  Using cached python_keycloak-2.12.0-py3-none-any.whl (52 kB)
  Attempting uninstall: rdflib
    Found existing installation: rdflib 6.3.2
    Uninstalling rdflib-6.3.2:
      Successfully uninstalled rdflib-6.3.2
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: python-keycloak
    Found existing installation: python-keycloak 1.4.0
    Uninstalling python-keycloak-1.4.0:
      Successfully uninstalled python-keycloak-1.4.0
  Attempting uninstall: marketplace-sdk
    Found existing installation: marketplace-sdk 0.4.0
    Uninstalling marketplace-sdk-0.4.0:
      Successfully uninstalled marketplace-sdk-0.4.0
  Consider adding this directory t

Import the MPSession for the data sink client and the custom requests-module

In [25]:
import requests
import tempfile
from getpass import getpass
from pprint import pprint

from marketplace.datasink_client.session import MPSession
from osp.core.namespaces import emmo

Set the host of your reaxpro-platform. The IP address is the respective address where you launched the app via docker or singularity.

In [2]:
HOST = "172.18.57.64:8080"

Now, let us repeat the tutorial which is available from the [official documentation](https://reaxpro-workflow-service.readthedocs.io/en/latest/index.html), but in terms of the [RES calculation in ams](https://reaxpro-workflow-service.readthedocs.io/en/latest/usecases.html#res-calculation). 

In [3]:
response = requests.get(f"http://{HOST}/models/registered")
print("First of all, we make sure that the PES Calculation is our registered models:")
pprint(response.json())

First of all, we make sure that the PES Calculation is our registered models:
{'message': 'Fetched registry of data models.',
 'registered_models': ['COPt111FullscaleModel',
                       'COPt111FromMesoScaleModel',
                       'PESExploration',
                       'COPt111MesoscaleModel',
                       'COCatalyticFOAMModel',
                       'EnergyLandscapeRefinement',
                       'COpyZacrosModel']}


Now, we define in the input data from the documentation:

In [4]:
data = {
    "force_field": "CHONSFPtClNi",
    "solver_type": "Direct",
    "n_expeditions": 30,
    "n_explorers": 3,
    "max_energy": 2.0,
    "max_distance": 3.8,
    "random_seed": 100,
    "fixed_region": "surface",
    "reference_region": "surface",
    "symmetry_check": "T",
    "molecule": "4442d5c3-4b61-4b13-9bbb-fdf942776ca6",
    "lattice": "4442d5c3-4b61-4b13-9bbb-fdf942776ca6"
}    

In [5]:
response = requests.post(f"http://{HOST}/models/create/PESExploration", json=data)
print("Create the model and get the cache id from the response:")
cache = response.json()
pprint(cache)

Create the model and get the cache id from the response:
{'cache_id': '174fa4b7-2931-4dc2-8040-6b5acf4adfee'}


In [6]:
response = requests.post(f"http://{HOST}/task/send", json=cache)

print("Send the task and get the task id from the response:")

task = response.json()
pprint(task)
task_id = task["task_id"]

Send the task and get the task id from the response:
{'args': None,
 'date_done': None,
 'kwargs': None,
 'state': 'PENDING',
 'status': 'PENDING',
 'task_id': '2cc9cfab-c349-49b1-9664-47aa97e3dcdc',
 'traceback': None}


In [7]:
response = requests.get(f"http://{HOST}/task/log/{task_id}")

print("Get logging messages of the task:\n")
print(response.text)

Get logging messages of the task:

2023-08-28 23:56:50,299 - INFO - received cache_key 174fa4b7-2931-4dc2-8040-6b5acf4adfee
2023-08-28 23:56:50,690 - INFO - 293 CUDS objects have been added to CeleryWorkflowSession
2023-08-28 23:56:50,703 - INFO - 1 CUDS object has been updated in CeleryWorkflowSession
2023-08-28 23:56:50,715 - INFO - 0 CUDS objects have been deleted from CeleryWorkflowSession
2023-08-28 23:56:50,732 - INFO - Did not find any workflow steps with
            complementary workers. Will scan for single
            object of type emmo.Calculation.
2023-08-28 23:56:50,750 - INFO - Found additional workers [(<emmo.ProcessSearch: e7bb9e3a-35ee-4648-9ac2-41d82fc57d61,  CeleryWorkflowSession: @0x14a712df4fd0>, 'simphony-ams')] in the buffer,
        but will ignored because not part of a workflow chain.
2023-08-28 23:56:50,804 - INFO - received cache_key 174fa4b7-2931-4dc2-8040-6b5acf4adfee
2023-08-28 23:56:51,114 - INFO - 293 CUDS objects have been added to Some Wrapper Sessi

In [8]:
response = requests.get(f"http://{HOST}/task/status/{task_id}")

print("Check that the job was successful task:\n")
pprint(response.json())

Check that the job was successful task:

{'args': None,
 'date_done': '2023-08-28T21:56:54.112362',
 'kwargs': None,
 'state': 'SUCCESS',
 'status': 'SUCCESS',
 'task_id': '2cc9cfab-c349-49b1-9664-47aa97e3dcdc',
 'traceback': None}


In [9]:
response = requests.get(f"http://{HOST}/task/result/{task_id}")

print("Get results from the simulation:\n")
result = response.json()
pprint(result)

graph_key = result["result"]["cache_meta"]
raw_data_key = result["result"]["cache_raw"]["0_simphony-ams"]

Get results from the simulation:

{'date_done': '2023-08-28T21:56:54.112362',
 'result': {'cache_meta': '2527abbc-0ee0-421a-8dba-45d80ea6fc5a',
            'cache_raw': {'0_simphony-ams': '97876357-dcec-4230-9296-90c358379abb'}},
 'task_id': '2cc9cfab-c349-49b1-9664-47aa97e3dcdc',
 'traceback': None}


In [10]:
print("First of all, get the resulting graph from the simulation:")

response = requests.get(f"http://{HOST}/cache/download/{graph_key}")

with tempfile.NamedTemporaryFile("w", delete=False, suffix=".ttl") as tmp_graph:
    tmp_graph.write(response.text)
print("File name:", tmp_graph.name)

print("Then, get zipped archive with the raw data from the simulation:")

response = requests.get(f"http://{HOST}/cache/download/{raw_data_key}")

with tempfile.NamedTemporaryFile("w", delete=False, suffix=".tar") as tmp_raw:
    tmp_raw.write(response.text)
print("File name:", tmp_raw.name)

First of all, get the resulting graph from the simulation:
File name: /scratch-local/mbueschel.3552932/tmpla_wqhvn.ttl
Then, get zipped archive with the raw data from the simulation:
File name: /scratch-local/mbueschel.3552932/tmprkhbz9hj.tar


Now that we have down our data, we will interact with the data sink on the MarketPlace in order to store and organize our data.

For this purpose, we set our client ID (Make sure that you purchased it on the platform first).

In [11]:
client_id = "845f0022-fa5e-4ea2-a601-947c9f39be01"

And set the access token:

In [13]:
token = getpass()

 ········


And start the MP session:

In [14]:
session = MPSession(access_token=token, client_id=client_id)

In order to follow the conventions of DCAT (the data cataloge), we define a new collection name to which we upload our data. For simplicity, we will use the `task_id` as collection name.

In [32]:
print("Upload the graph:\n")

graph_name = f"graph-{task_id}"
raw_name = f"raw-data-ams-{task_id}"

objects = session.create_dataset_from_path(
        path=tmp_graph.name,
        dataset_name=graph_name,
        collection_name=task_id,
)
print(objects)

print("\nAnd upload the raw data:\n")

objects = session.create_dataset_from_path(
        path=tmp_raw.name,
        dataset_name=raw_name,
        collection_name=task_id,
)
print(objects)

Upload the graph:

Error: Server returned 400 while creating collection 2cc9cfab-c349-49b1-9664-47aa97e3dcdc: {"detail":"There is already a collection with given name. Root collection name should be always unique."}
None

And upload the raw data:

Error: Server returned 400 while creating collection 2cc9cfab-c349-49b1-9664-47aa97e3dcdc: {"detail":"There is already a collection with given name. Root collection name should be always unique."}
None


In [31]:
objects = session.get_collection_dcat(collection_name=task_id)
print(objects)

API encountered exception. Please check if all your environment variables configured properly once. Error details:  The app does not support this capability.
None


And let us send a SPARQL-query to a graph in the data sink:

Give me all calculation individuals?

In [33]:
query = f"""PREFIX skos:<http://www.w3.org/2004/02/skos/core#>

    SELECT ?calculation ?calculationname WHERE {{
        ?calculationtype rdfs:subClassOf* <{emmo.Calculation.iri}> .
        ?calculation rdf:type ?calculationtype .
        ?calculationtype skos:prefLabel ?calculationname .
    }}
"""

objects = session.query_dataset(collection_name=task_id, dataset_name=graph_name, query=query)
pprint(objects)



API encountered exception. Please check if all your environment variables configured properly once. Error details:  The app does not support this capability.
None


Give all calculations with output TotalEnelectronic Energy?

In [34]:
query = f"""PREFIX skos:<http://www.w3.org/2004/02/skos/core#>

    SELECT ?calculation ?calculationname ?optgeometry ?valuenumber  ?unitsymbol WHERE {{
        ?calculationtype rdfs:subClassOf* <{emmo.Calculation.iri}> .
        ?calculation rdf:type ?calculationtype .
        ?calculationtype skos:prefLabel ?calculationname .
        ?calculation <{emmo.hasOutput.iri}> ?optgeometry .
        ?optgeometry <{emmo.hasProperty.iri}> ?totalelectronicenergy .
        ?totalelectronicenergy rdf:type <{emmo.TotalElectronicEnergy.iri}> .
        ?totalelectronicenergy <{emmo.hasQuantityValue.iri}> ?value .
        ?value <{emmo.hasNumericalData.iri}> ?valuenumber .
        ?totalelectronicenergy <{emmo.hasReferenceUnit.iri}> ?unit .
        ?unit <{emmo.hasSymbolData.iri}> ?unitsymbol .

    }}
"""

objects = session.query_dataset(collection_name=task_id, dataset_name=graph_name, query=query)
pprint(objects)

API encountered exception. Please check if all your environment variables configured properly once. Error details:  The app does not support this capability.
None
